In [ ]:
'''
Pseudocode for the conversation between a planner and an assistant to better reason about a problem and provide a solution.

user_message = "<user_message>"
planner = Planner(user_message)
assistant = Assistant(user_message)

task_id = 0
turn_id = 0

task = None
assistant_response = None

assistant_thoughts = []


while True:

    # 1.0 Update next task
    task = planner.continue_planning(task, assistant_thoughts)

    if "Conclusion: " in task: # If the planner decides to conclude, then the conversation ends and we return the task / conclusion
        break
    
    # 2.0 Assistant executes next task
    task_id += 1    
    assistant_response = assistant.continue_thinking(task, assistant_thoughts)
    assistant_thoughts.extend(assistant_response.split("\n")) # Split on new lines

class Planner:
    def init(self, user_message):
        self.user_message = user_message
        self.system_prompt = "You are a planner for an assistant trying to solve a user task. Your task is to help refine the assistant's reasoning by generating the next task for the assistant to think about. You will be given the user problem and the assistant's chain of thought so far."

    def continue_planning(self, task, assistant_thoughts):
        user_prompt = f"USER TASK: {task}\n\nASSISTANT CURRENT CHAIN-OF-THOUGHT:\n"
        if assistant_thoughts:
            user_prompt += "\n".join(assistant_thoughts)
        messages=[
            {"role": "system", "content": self.system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                ],
            },
        ],
        return generate(messages)

class Assistant:
    def init(self, user_message):
        self.user_message = user_message
        self.system_prompt = "You are a helpful assistant."

    def continue_thinking(self, task):
        pass
'''

In [10]:
from openai import OpenAI
import time

class Model:
    def __init__(self):
        openai_api_key = "EMPTY"
        openai_api_base = "https://ray-stable-killdeer.ngrok-free.app/v1"
        self.client = OpenAI(
            api_key=openai_api_key,
            base_url=openai_api_base,
        )

    def generate(self, messages):
        time_start = time.time()
        chat_response = self.client.chat.completions.create(
            model="llama-3-1-8b-instruct",
            messages=messages,
            temperature=0.0,
            n=1,
            max_tokens=1024
        )

        print("Time taken:", time.time() - time_start)
        print("\n\nChat response text:", chat_response.choices[0].message.content)

        return chat_response.choices[0].message.content
    
model = Model()
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "text", "text": '''oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz'''},
        ],
    },
]
model.generate(messages)

Time taken: 4.7769856452941895


Chat response text: To decode the given text, let's analyze the original example: "oyfjdnisdr rtqwainr acxz mynzbhhx". 

It seems like the letters are shifted in a specific pattern. To determine the pattern, let's look for any repeating letters or sequences.

Upon closer inspection, it appears that the letters are shifted by a fixed number of positions in the alphabet. This is a common technique used in cryptography, known as a Caesar cipher.

To decode the text, we need to find the shift value. Let's try to find a word or sequence that is recognizable in the original example.

One possible approach is to look for common letter combinations or words that are often used in language. 

After analyzing the text, I found a possible shift value. If we apply this shift value to the original example, we can decode the text.

Here's the decoded text for the original example: "oyfjdnisdr rtqwainr acxz mynzbhhx" -> "think step by step".

Now, let's apply the same

'To decode the given text, let\'s analyze the original example: "oyfjdnisdr rtqwainr acxz mynzbhhx". \n\nIt seems like the letters are shifted in a specific pattern. To determine the pattern, let\'s look for any repeating letters or sequences.\n\nUpon closer inspection, it appears that the letters are shifted by a fixed number of positions in the alphabet. This is a common technique used in cryptography, known as a Caesar cipher.\n\nTo decode the text, we need to find the shift value. Let\'s try to find a word or sequence that is recognizable in the original example.\n\nOne possible approach is to look for common letter combinations or words that are often used in language. \n\nAfter analyzing the text, I found a possible shift value. If we apply this shift value to the original example, we can decode the text.\n\nHere\'s the decoded text for the original example: "oyfjdnisdr rtqwainr acxz mynzbhhx" -> "think step by step".\n\nNow, let\'s apply the same shift value to the given text: "

In [9]:
from openai import OpenAI
import time

class Model:
    def __init__(self):
        openai_api_key = "EMPTY"
        openai_api_base = "https://ray-stable-killdeer.ngrok-free.app/v1"
        self.client = OpenAI(
            api_key=openai_api_key,
            base_url=openai_api_base,
        )

    def generate(self, messages, temperature=0.0):
        time_start = time.time()
        chat_response = self.client.chat.completions.create(
            model="llama-3-1-8b-instruct",
            messages=messages,
            temperature=temperature,
            n=1,
            max_tokens=1024
        )

        print("Time taken:", time.time() - time_start)
        print("\n\nChat response text:", chat_response.choices[0].message.content)

        return chat_response.choices[0].message.content

class Planner:
    def __init__(self, user_message, model):
        self.user_message = user_message
        self.system_prompt = f'''You are a planner for an assistant trying to solve a user query. You will be given the user query and the assistant's chain of thought so far.

Your task is to 1) rate the assistant's latest thought so far and rate it as '+' (great), '0' (okay), or '-' (bad), and 2) help refine the assistant's next thoughts by generating the next task for the assistant to think about to maximize the probability that the assistant's final answer is correct and minimize risks of the assistant's final answer being incorrect.

A Great '+' thought is if both the reasoning and action in the response a good assistant would make in that turn again. Most of the time it's a clear cut step forward towards completing the user task, includes thoughtful reasoning, and the action executes as expected and desired.

An Okay '0' thought is anything that's reasonable for an assistant to reason and act, but it's not offering any insight in its reasoning, doesn't further the task progress by exploring an approach, performing an action, or offering an idea for the next step. This could be that the reasoning is sound and the action is a good but incomplete first step, so you need to wait for the next turns' actions and screenshots to tell if it was a good or bad turn. If you are unsure if the turn is good or bad, especially when there isn't a trajectory evaluation provided nor more screenshots after the (CURRENT EVALUATION POST-ACTION TURN) turn to tell for sure if the action ended up being good or bad, then rate it as '0'.

A Bad '-' thought is one that is obviously incorrect - the assistant repeats the same mistakes that aren't working instead of trying a different approach, confidently reasons and acts incorrectly, is off-topic/weird, leads the trajectory into a clear dead-end, or the action doesn't execute as expected.

Please respond in the following format:
(CURRENT EVALUATION TURN) Assistant Turn Response: <restate what the (CURRENT EVALUATION TURN) Assistant Turn Response is>
Observations: <a) describe the (CURRENT EVALUATION TURN) Screenshot Turn, b) describe the (CURRENT EVALUATION POST-ACTION TURN) Screenshot Turn, c) describe the subsequent Screenshot Turns after (CURRENT EVALUATION POST-ACTION TURN) to gather more information about the actual effect of the assistant turn response, and then d) analyze precisely what changes happened between the (CURRENT EVALUATION TURN) Screenshot Turn and (CURRENT EVALUATION POST-ACTION TURN) Screenshot Turn to figure out exactly what effect the assistant turn response action actually had>
Reasoning: <according to the observations, reason what would be an appropriate rating and why. when appropriate, also reason what the assistant should do (ex. if it keeps repeating the same element interactions and actions that aren't working, it should try a different approach and not repeat the same element interactions and actions [do not suggest to find better coordinates, but suggest the assistant to find a different approach]. if it doesn't know how to do something, it should try to Google for help.)>
Rating: <based on the reasoning result, provide a rating of '+', '0', or '-', in the format of '+', '0', or '-' in a single line to reflect the turn rating.>'''
        
        
#         f'''You are a turn-by-turn evaluator and you are judging an assistant's (CURRENT EVALUATION TURN) Assistant Turn response in its trajectory so far in trying to complete a user task.

# The user will give you 1) a user task and 2) sequential turns of assistant code actions and screenshots (up to the last {max_images} screenshots) of the assistant trying to complete the user task so far. The assistant turn that you must evaluate will be labeled as "(CURRENT EVALUATION TURN)", with the screenshot turn that it's acting on labeled as "(CURRENT EVALUATION TURN)" and the effect of the assistant turn labeled as "(CURRENT EVALUATION POST-ACTION TURN)".

# Your task is to analyze the (CURRENT EVALUATION TURN) Assistant Turn Response correctness level and rate it as '+' (great), '0' (okay), or '-' (bad). 

# A Great '+' turn is if both the reasoning and action in the response a good assistant would make in that turn again. Most of the time it's a clear cut step forward towards completing the user task, includes thoughtful reasoning, and the action executes as expected and desired.

# An Okay '0' turn is anything that's reasonable for an assistant to reason and act, but it's not offering any insight in its reasoning, doesn't further the task progress by exploring an approach, performing an action, or offering an idea for the next step. This could be that the reasoning is sound and the action is a good but incomplete first step, so you need to wait for the next turns' actions and screenshots to tell if it was a good or bad turn. If you are unsure if the turn is good or bad, especially when there isn't a trajectory evaluation provided nor more screenshots after the (CURRENT EVALUATION POST-ACTION TURN) turn to tell for sure if the action ended up being good or bad, then rate it as '0'.

# A Bad '-' turn is one that is obviously incorrect - the assistant repeats the same mistakes that aren't working instead of trying a different approach, confidently reasons and acts incorrectly, is off-topic/weird, leads the trajectory into a clear dead-end, or the action doesn't execute as expected.

# Please respond in the following format:
# (CURRENT EVALUATION TURN) Assistant Turn Response: <restate what the (CURRENT EVALUATION TURN) Assistant Turn Response is>
# Observations: <a) describe the (CURRENT EVALUATION TURN) Screenshot Turn, b) describe the (CURRENT EVALUATION POST-ACTION TURN) Screenshot Turn, c) describe the subsequent Screenshot Turns after (CURRENT EVALUATION POST-ACTION TURN) to gather more information about the actual effect of the assistant turn response, and then d) analyze precisely what changes happened between the (CURRENT EVALUATION TURN) Screenshot Turn and (CURRENT EVALUATION POST-ACTION TURN) Screenshot Turn to figure out exactly what effect the assistant turn response action actually had>
# Reasoning: <according to the observations, reason what would be an appropriate rating and why. when appropriate, also reason what the assistant should do (ex. if it keeps repeating the same element interactions and actions that aren't working, it should try a different approach and not repeat the same element interactions and actions [do not suggest to find better coordinates, but suggest the assistant to find a different approach]. if it doesn't know how to do something, it should try to Google for help.)>
# Rating: <based on the reasoning result, provide a rating of '+', '0', or '-', in the format of '+', '0', or '-' in a single line to reflect the turn rating.>'''
        
#         '''You are a planner for an assistant trying to solve a user query. Your task is to help refine the assistant's reasoning by generating the next task for the assistant to think about. You will be given the user query and the assistant's chain of thought so far.
        
# Respond with just the next task for the assistant to think about.'''
        
        self.model = model

    def continue_planning(self, assistant_thoughts):
        user_prompt = f"USER QUERY: {self.user_message}\n\nASSISTANT CURRENT CHAIN-OF-THOUGHT: "
        if assistant_thoughts:
            user_prompt += "\n".join(assistant_thoughts)
        else:
            user_prompt += "''"
        print("Planner user prompt:", user_prompt)

        messages=[
            {"role": "system", "content": self.system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                ],
            },
        ]

        planner_response = self.model.generate(messages)

        print(f"\nPlanner response: {planner_response}")

        return planner_response


user_message = '''oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz'''

model = Model()
planner = Planner(user_message=user_message, model=model)

task_id = 0
turn_id = 0

task = None
assistant_response = None

assistant_thoughts = []

# DEBUG: Try with the first thoughts
assistant_thoughts.append("To decode the given ciphertext using the example provided, we'll use the pattern identified in the example.")

# 1.0 Update next task
task = planner.continue_planning(assistant_thoughts)
print(f"\nNext task: {task}")

Planner user prompt: USER QUERY: oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz

ASSISTANT CURRENT CHAIN-OF-THOUGHT: To decode the given ciphertext using the example provided, we'll use the pattern identified in the example.
Time taken: 0.5964682102203369


Chat response text: Identify the pattern used in the example to shift or replace characters, and apply it to the new ciphertext to determine the decoding method.

Planner response: Identify the pattern used in the example to shift or replace characters, and apply it to the new ciphertext to determine the decoding method.

Next task: Identify the pattern used in the example to shift or replace characters, and apply it to the new ciphertext to determine the decoding method.


In [36]:
# Quick test of what if it's just two Assistant's continuing its thoughts

'''
Training data:


Inference time:
<user>Steps allowed: <# of turns>\n\n{user_prompt}</user>
<assistant> 
    <step_1>thought 1</step_1> 
    <step_2>thought 2</step_2>
    <step_3>thought 3</step_3>
    ...
    <step_n>thought n</step_n>
</assistant> (we should SFT for this format, and then for every new line, we can just prepend and append the step tags)

Q1: What does this even gain though? How is this different than the current GPT-4o / LLaMA generation format?
1. You have steps that you can control for how many steps the assistant thinks for?
2. Then you can just run RL / Proximal Policy Optimization (PPO) / Grader on the final trajectory, running the trajectory with temp=1 and sample like 100 trajectories?

Q2: Why doesn't current PPO already achieve this? (or maybe it does with Reinforcement Finetuning?)
1. With temperature = 1, you should be able to achieve this?


Q3: Can we show inference results by controlling max tokens parameter?
- Quick experiment to run
'''

from openai import OpenAI
import time

class Model:
    def __init__(self):
        openai_api_key = "EMPTY"
        openai_api_base = "https://ray-stable-killdeer.ngrok-free.app/v1"
        self.client = OpenAI(
            api_key=openai_api_key,
            base_url=openai_api_base,
        )

    def generate(self, messages, temperature=0.0, stop="\n"):
        time_start = time.time()
        chat_response = self.client.chat.completions.create(
            model="llama-3-1-8b-instruct",
            messages=messages,
            temperature=temperature,
            n=1,
            max_tokens=1024,
            stop=stop
        )

        print("Time taken:", time.time() - time_start)
        print("\n\nChat response text:", chat_response.choices[0].message.content)

        return chat_response.choices[0].message.content

class Assistant:
    def __init__(self, user_message, model):
        self.user_message = user_message
        self.system_prompt = '''You are a helpful assistant.
        
You will be given a user prompt and chain-of-thought. 

Your task is to refine and continue the chain-of-thought.

Once you reach a final answer with the chain-of-thought, end with "<submit/>" to submit the entire chain-of-thought as the final full response to the user prompt.

Respond with only the next thought.''' # TODO: maybe train a model to generate the next thought in this format?
        self.model = model

    def continue_thinking(self, assistant_thoughts):
        user_prompt = f"PROMPT: {self.user_message}\n\nCHAIN-OF-THOUGHT: "
        if assistant_thoughts:
            user_prompt += "\n".join(assistant_thoughts)
        print("Prompt:\n", user_prompt, "\n\n")

        messages=[
            {"role": "system", "content": self.system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                ],
            },
        ]
        return self.model.generate(messages)


user_message = '''oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz'''

model = Model()
assistant = Assistant(user_message=user_message, model=model)

turn_id = 0

task = None

assistant_thoughts = []

# Generate next thought
for i in range(2):
    print(f"\n--- Turn {i+1} ---\n")
    thought = assistant.continue_thinking(assistant_thoughts)
    assistant_thoughts.append(thought)
    print(f"Thought: {thought}")


--- Turn 1 ---

Prompt:
 PROMPT: oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz

CHAIN-OF-THOUGHT:  


Time taken: 2.0613152980804443


Chat response text: To start decoding, I'll first look for any patterns or shifts in the letters. The given example "oyfjdnisdr rtqwainr acxz mynzbhhx" seems to have a mix of letters, but there's no immediate pattern. 
Thought: To start decoding, I'll first look for any patterns or shifts in the letters. The given example "oyfjdnisdr rtqwainr acxz mynzbhhx" seems to have a mix of letters, but there's no immediate pattern. 

--- Turn 2 ---

Prompt:
 PROMPT: oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz

CHAIN-OF-THOUGHT: To start decoding, I'll first look for any patterns or shifts in the letters. The given example "oyfjdnisdr rtqwainr acxz mynzbh

KeyboardInterrupt: 

In [16]:
# Sample inference code for calling LLaMA-3.2-1B-Instruct model.
from openai import OpenAI
import time

user_prompt = '''oyfjdnisdr rtqwainr acxz mynzbhhx -> Think step by step

Use the example above to decode:

oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz'''

openai_api_key = "EMPTY"
openai_api_base = "https://ray-stable-killdeer.ngrok-free.app/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

time_start = time.time()
chat_response = client.chat.completions.create(
    model="llama-3-2-1b-instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
            ],
        },
    ],
    temperature=0.0,
    n=1,
    max_tokens=1024
)

print("Time taken:", time.time() - time_start)

print("\n\nChat response text:", chat_response.choices[0].message.content)

Time taken: 3.3868584632873535


Chat response text: To decode the message, let's break it down step by step:

1. Start with the first letter of each word:
   - o = o
   - y = y
   - e = e
   - k = k
   - a = a
   - i = i
   - j = j
   - z = z
   - d = d
   - f = f
   - a = a
   - p = p
   - a = a
   - t = t
   - c = c
   - g = g
   - s = s
   - u = u
   - a = a
   - k = k
   - y = y
   - b = b
   - h = h
   - a = a
   - i = i
   - r = r
   - a = a
   - c = c
   - h = h
   - b = b
   - h = h
   - a = a
   - i = i
   - r = r
   - a = a
   - c = c
   - d = d
   - f = f
   - o = o
   - u = u
   - l = l
   - l = l
   - x = x
   - z = z

2. Combine the decoded letters to form the words:
   - o = o
   - y = y
   - e = e
   - k = k
   - a = a
   - i = i
   - j = j
   - z = z
   - d = d
   - f = f
   - a = a
   - p = p
   - a = a
   - t = t
   - c = c
   - g = g
   - s = s
   - u = u
   - a = a
   - k = k
   - y = y
   - b = b
   - h = h
   - a = a
   - i = i
   - r = r
   - a = a
   - c = c
 